In [1]:
from collections import defaultdict, Counter
from intervaltree import IntervalTree
import itertools
import functools
import numpy as np
import re
from typing import Callable, TypeVar

T = TypeVar('T')

def data(day: int, parser: Callable[[str], T] = str) -> list[T]:
  with open(f"./data/day{day}.txt") as f:
    return [parser(line.strip()) for line in f.readlines()]

In [99]:
data1 = data(1)

def day1(match):
    sum = 0
    for line in data1:
        matched = match(line)
        first, last = matched[0], matched[-1]
        # print(first, last, sum)
        sum += int(first + last)
    return sum

print(day1(lambda x: re.findall(r'\d', x)))

def part2match(x):
    change = {
        'one': '1',
        'two': '2',
        'three': '3',
        'four': '4',
        'five': '5',
        'six': '6',
        'seven': '7',
        'eight': '8',
        'nine': '9',
    }
    matched = re.findall(r'(?=(\d|one|two|three|four|five|six|seven|eight|nine))', x)
    return [change[match] if match in change else match for match in matched]

day1(part2match)
    

55816


54980

In [148]:
def format2(line):
    def tuplefy(round):
        result = [0,0,0]
        for i in round:
            result[0 if 'blue' in i else 1 if 'green' in i else 2] += int(i.split(' ')[0])
        return result

    line = line.split(': ')[-1]
    rounds = line.split('; ' )
    return np.array([tuplefy(x.split(', ')) for x in rounds])

data2 = data(2, format2)

def day2_1(start):
    result = 0
    for i, game in enumerate(data2):
        possible = np.subtract(start, game)
        if np.any(possible < 0):
            continue
        result += i+1
    return result

print(day2_1((14, 13, 12)))

def day2_2():
    result = 0
    for game in data2:
        requirements = np.amax(game, axis=0)
        result += functools.reduce(lambda x, y: x*y, requirements, 1)
    return result
day2_2()

2377


71220

In [130]:
data3 = data(3)

def day3():
    numbers = []
    for i, line in enumerate(data3):
        numbers += [(m.group(), (i, m.start())) for m in re.finditer(r'\d+', line)]

    array3 = np.pad(np.array([list(line) for line in data3]), ((1, 1), (1, 1)), constant_values='.')
    gears = defaultdict(lambda: [])

    part1 = 0
    for n, (i, j) in numbers:
        sliced = array3[i:i+3, j:j+2+len(n)]
        stringified =  ''.join(''.join(x) for x in sliced)
        if re.search(r'[^\d.]', stringified):
            part1 += int(n)
        if stars := np.where(sliced == '*'):
            for x, _ in enumerate(stars[0]):
                gears[(i+stars[0][x], j+stars[1][x])].append(int(n))

    part2 = 0
    for g in gears:
        if len(gears[g]) == 2:
            part2 += gears[g][0] * gears[g][1]

    return part1, part2

day3()

(521515, 69527306)

In [5]:
def parse4(line):
    winning, have = map(
        lambda numbers: {int(x) for x in numbers.split()},
        line.split(':')[-1].split('|')
    )
    return winning, have

data4 = data(4, parse4)

def day4(data):
    result, cards = 0, Counter()
    for i, line in enumerate(data):
        matches = line[0].intersection(line[1])
        if matches:
            result += 2**(len(matches)-1)
        for n in range(len(matches)):
            cards[i+1+n] += cards[i]+1
    return result, cards.total() + len(data)

day4(data4)


(25651, 19499881)

In [67]:
def create_map(lines):
    name = lines[0].split(' ')[0]
    ranges = IntervalTree()
    for i, line in enumerate(lines[1:]):
        if not line:
            break
        dest, source, size = map(int, line.split())
        ranges[source:source+size] = dest
    return name, ranges, lines[i+2:]

def day5_maps(data):
    seeds, data = map(int, data[0].split(': ')[-1].split()), data[2:]
 
    maps = []
    while len(data):
        name, intervals, data = create_map(data)
        maps.append(intervals)
    print(maps)

    return list(seeds), maps

seeds, data5 = day5_maps(data(5))


[IntervalTree([Interval(50, 98, 52), Interval(98, 100, 50)]), IntervalTree([Interval(0, 15, 39), Interval(15, 52, 0), Interval(52, 54, 37)]), IntervalTree([Interval(0, 7, 42), Interval(7, 11, 57), Interval(11, 53, 0), Interval(53, 61, 49)]), IntervalTree([Interval(18, 25, 88), Interval(25, 95, 18)]), IntervalTree([Interval(45, 64, 81), Interval(64, 77, 68), Interval(77, 100, 45)]), IntervalTree([Interval(0, 69, 1), Interval(69, 70, 0)]), IntervalTree([Interval(56, 93, 60), Interval(93, 97, 56)])]


In [75]:
a = IntervalTree()
not a.at(1)
a[1:5] = 10
print(a)
a.at(5)

IntervalTree([Interval(1, 5, 10)])


set()

In [80]:
78-50

28

In [79]:
data5[0].at(79)

{Interval(50, 98, 52)}

In [84]:
for seed in seeds:
    # print(seed)
    for interval in data5:
        if not (hit := interval.at(seed)):
            # print(seed, hit)
            continue
        # print(seed, hit)
        start, end, dest = next(iter(hit))
        # print(seed, start, dest)
        seed = (seed-start)+dest
    print(seed)

79 50 52
81 25 18
74 64 68
78 56 60
82
14 0 39
53 53 49
49 25 18
42 0 1
43
55 50 52
57 53 49
53 25 18
46 45 81
82 56 60
86
13 0 39
52 11 0
41 25 18
34 0 1
35
